In [3]:
import pandas as pd
import numpy as np
import sys
import os

# Add project root to path
sys.path.append(r"D:\data-problem-solving")

from db import load_table, engine

ad_update_advertiser =  load_table('ad_update_advertiser')
ad_update_daily_pay =  load_table('ad_update_daily_pay')

In [5]:
# Step 1: Merge advertiser info with daily payments (LEFT JOIN)
df = ad_update_advertiser.merge(ad_update_daily_pay, on='user_id', how="left")

# Step 2: Define conditions for transitions
conditions = [
    (df["status"] == "NEW") & (df["paid"].notna()),
    (df["status"] == "NEW") & (df["paid"].isna()),
    (df["status"] == "EXISTING") & (df["paid"].notna()),
    (df["status"] == "EXISTING") & (df["paid"].isna()),
    (df["status"] == "CHURN") & (df["paid"].notna()),
    (df["status"] == "CHURN") & (df["paid"].isna()),
    (df["status"] == "RESURRECT") & (df["paid"].notna()),
    (df["status"] == "RESURRECT") & (df["paid"].isna())
]

# Step 3: Define the corresponding new statuses
choices = [
    "EXISTING",  # NEW + paid
    "CHURN",     # NEW + no payment
    "EXISTING",  # EXISTING + paid
    "CHURN",     # EXISTING + no payment
    "RESURRECT", # CHURN + paid
    "CHURN",     # CHURN + no payment
    "EXISTING",  # RESURRECT + paid
    "CHURN"      # RESURRECT + no payment
]

df['new_status'] = np.select(conditions, choices, default=df["status"])

result = df[['user_id', 'new_status']].sort_values('user_id')

print(result)

   user_id new_status
2  alibaba   EXISTING
0     bing      CHURN
1    yahoo   EXISTING
